<a href="https://colab.research.google.com/github/JEEVANnr2001/machine-learning-Projects/blob/main/movie_ticket_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from lightgbm import LGBMRegressor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.svm import LinearSVR

In [ ]:
df = pd.read_csv("cinemaTicket_Ref.csv")
df

,film_code,cinema_code,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,1492,304,3900000,26,0.0,4.0,4.26,150000.0,26.0,610.328638,2018-05-05,5.0,2.0,5.0
1,1492,352,3360000,42,0.0,5.0,8.08,80000.0,42.0,519.801980,2018-05-05,5.0,2.0,5.0
2,1492,489,2560000,32,0.0,4.0,20.00,80000.0,32.0,160.000000,2018-05-05,5.0,2.0,5.0
3,1492,429,1200000,12,0.0,1.0,11.01,100000.0,12.0,108.991826,2018-05-05,5.0,2.0,5.0
4,1492,524,1200000,15,0.0,3.0,16.67,80000.0,15.0,89.982004,2018-05-05,5.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27036,1499,144,1200000,15,0.0,8.0,3.68,80000.0,15.0,407.608696,2018-07-12,7.0,3.0,12.0
27037,1499,323,1140000,19,0.0,5.0,4.37,60000.0,19.0,434.782609,2018-07-12,7.0,3.0,12.0
27038,1499,154,1120000,16,0.0,4.0,1.33,70000.0,16.0,1203.007519,2018-07-12,7.0,3.0,12.0
27039,1499,480,1120000,16,0.0,2.0,7.27,70000.0,16.0,220.082531,2018-07-12,7.0,3.0,12.0


In [ ]:
df = df.drop(['film_code','cinema_code'],axis=1)

In [ ]:
df.isnull().sum()

total_sales     0
tickets_sold    0
tickets_out     1
show_time       1
occu_perc       8
ticket_price    1
ticket_use      1
capacity        8
date            1
month           1
quarter         1
day             1
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
for col in df.columns:
    print(col, ":", df[col].unique().shape[0])

total_sales : 2933
tickets_sold : 760
tickets_out : 33
show_time : 20
occu_perc : 5205
ticket_price : 4823
ticket_use : 771
capacity : 13182
date : 229
month : 9
quarter : 4
day : 31


In [ ]:
columns_to_drop = [col for col in df.columns if df[col].unique().shape[0] == 1]
df.drop(columns=columns_to_drop, inplace=True)
df.head()

,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,3900000,26,0.0,4.0,4.26,150000.0,26.0,610.328638,2018-05-05,5.0,2.0,5.0
1,3360000,42,0.0,5.0,8.08,80000.0,42.0,519.801980,2018-05-05,5.0,2.0,5.0
2,2560000,32,0.0,4.0,20.00,80000.0,32.0,160.000000,2018-05-05,5.0,2.0,5.0
3,1200000,12,0.0,1.0,11.01,100000.0,12.0,108.991826,2018-05-05,5.0,2.0,5.0
4,1200000,15,0.0,3.0,16.67,80000.0,15.0,89.982004,2018-05-05,5.0,2.0,5.0


TO Find The Correlation between features


In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,month,quarter,day
total_sales,1.000000,0.915786,0.027682,0.341549,0.477717,0.321220,0.915587,0.194826,-0.167926,-0.133251,-0.004201
tickets_sold,0.915786,1.000000,0.025959,0.358325,0.544567,0.118789,0.999855,0.236229,-0.167597,-0.128990,0.005238
tickets_out,0.027682,0.025959,1.000000,0.041875,0.019427,0.022386,0.008937,0.012904,-0.013806,-0.016048,0.009915
show_time,0.341549,0.358325,0.041875,1.000000,-0.014934,0.143557,0.357718,0.642865,-0.107529,-0.119017,-0.010265
occu_perc,0.477717,0.544567,0.019427,-0.014934,1.000000,0.099215,0.544398,-0.217176,-0.114260,-0.059468,0.008909
ticket_price,0.321220,0.118789,0.022386,0.143557,0.099215,1.000000,0.118443,-0.064537,-0.031630,-0.030225,-0.037871
ticket_use,0.915587,0.999855,0.008937,0.357718,0.544398,0.118443,1.000000,0.236079,-0.167412,-0.128755,0.005071
capacity,0.194826,0.236229,0.012904,0.642865,-0.217176,-0.064537,0.236079,1.000000,-0.044716,-0.061820,-0.006095
month,-0.167926,-0.167597,-0.013806,-0.107529,-0.114260,-0.031630,-0.167412,-0.044716,1.000000,0.885246,-0.227673
quarter,-0.133251,-0.128990,-0.016048,-0.119017,-0.059468,-0.030225,-0.128755,-0.061820,0.885246,1.000000,-0.254451


In [ ]:
df.drop(['day','capacity'],axis=1)



,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,date,month,quarter
0,3900000,26,0.0,4.0,4.26,150000.0,26.0,2018-05-05,5.0,2.0
1,3360000,42,0.0,5.0,8.08,80000.0,42.0,2018-05-05,5.0,2.0
2,2560000,32,0.0,4.0,20.00,80000.0,32.0,2018-05-05,5.0,2.0
3,1200000,12,0.0,1.0,11.01,100000.0,12.0,2018-05-05,5.0,2.0
4,1200000,15,0.0,3.0,16.67,80000.0,15.0,2018-05-05,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...
27035,1200000,20,0.0,3.0,2.75,60000.0,20.0,2018-07-12,7.0,3.0
27036,1200000,15,0.0,8.0,3.68,80000.0,15.0,2018-07-12,7.0,3.0
27037,1140000,19,0.0,5.0,4.37,60000.0,19.0,2018-07-12,7.0,3.0
27038,1120000,16,0.0,4.0,1.33,70000.0,16.0,2018-07-12,7.0,3.0


In [ ]:
df.drop(['ticket_price'],axis=1)

,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_use,capacity,date,month,quarter,day
0,3900000,26,0.0,4.0,4.26,26.0,610.328638,2018-05-05,5.0,2.0,5.0
1,3360000,42,0.0,5.0,8.08,42.0,519.801980,2018-05-05,5.0,2.0,5.0
2,2560000,32,0.0,4.0,20.00,32.0,160.000000,2018-05-05,5.0,2.0,5.0
3,1200000,12,0.0,1.0,11.01,12.0,108.991826,2018-05-05,5.0,2.0,5.0
4,1200000,15,0.0,3.0,16.67,15.0,89.982004,2018-05-05,5.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
27035,1200000,20,0.0,3.0,2.75,20.0,727.272727,2018-07-12,7.0,3.0,12.0
27036,1200000,15,0.0,8.0,3.68,15.0,407.608696,2018-07-12,7.0,3.0,12.0
27037,1140000,19,0.0,5.0,4.37,19.0,434.782609,2018-07-12,7.0,3.0,12.0
27038,1120000,16,0.0,4.0,1.33,16.0,1203.007519,2018-07-12,7.0,3.0,12.0


In [ ]:
df=df.drop(['ticket_price_final'],axis=1)
df.head()

,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day,ticket_price final
0,0.00390,26,0,4,4.26,0.15,26,610.328638,2018-05-05,5,2,5,150.0
1,0.00336,42,0,5,8.08,0.08,42,519.801980,2018-05-05,5,2,5,80.0
2,0.00256,32,0,4,20.00,0.08,32,160.000000,2018-05-05,5,2,5,80.0
3,0.00120,12,0,1,11.01,0.10,12,108.991826,2018-05-05,5,2,5,100.0
4,0.00120,15,0,3,16.67,0.08,15,89.982004,2018-05-05,5,2,5,80.0


In [ ]:
df.drop(['total_sales'],axis=1)

,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day
0,26,0.0,4.0,4.26,150000.0,26.0,610.328638,2018-05-05,5.0,2.0,5.0
1,42,0.0,5.0,8.08,80000.0,42.0,519.801980,2018-05-05,5.0,2.0,5.0
2,32,0.0,4.0,20.00,80000.0,32.0,160.000000,2018-05-05,5.0,2.0,5.0
3,12,0.0,1.0,11.01,100000.0,12.0,108.991826,2018-05-05,5.0,2.0,5.0
4,15,0.0,3.0,16.67,80000.0,15.0,89.982004,2018-05-05,5.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
27035,20,0.0,3.0,2.75,60000.0,20.0,727.272727,2018-07-12,7.0,3.0,12.0
27036,15,0.0,8.0,3.68,80000.0,15.0,407.608696,2018-07-12,7.0,3.0,12.0
27037,19,0.0,5.0,4.37,60000.0,19.0,434.782609,2018-07-12,7.0,3.0,12.0
27038,16,0.0,4.0,1.33,70000.0,16.0,1203.007519,2018-07-12,7.0,3.0,12.0


In [ ]:
# df['new_ticket_price'] = df['ticket_price']/1000

In [ ]:
df

,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day,new_ticket_price
0,3900000,26,0.0,4.0,4.26,150000.0,26.0,610.328638,2018-05-05,5.0,2.0,5.0,150.0
1,3360000,42,0.0,5.0,8.08,80000.0,42.0,519.801980,2018-05-05,5.0,2.0,5.0,80.0
2,2560000,32,0.0,4.0,20.00,80000.0,32.0,160.000000,2018-05-05,5.0,2.0,5.0,80.0
3,1200000,12,0.0,1.0,11.01,100000.0,12.0,108.991826,2018-05-05,5.0,2.0,5.0,100.0
4,1200000,15,0.0,3.0,16.67,80000.0,15.0,89.982004,2018-05-05,5.0,2.0,5.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27035,1200000,20,0.0,3.0,2.75,60000.0,20.0,727.272727,2018-07-12,7.0,3.0,12.0,60.0
27036,1200000,15,0.0,8.0,3.68,80000.0,15.0,407.608696,2018-07-12,7.0,3.0,12.0,80.0
27037,1140000,19,0.0,5.0,4.37,60000.0,19.0,434.782609,2018-07-12,7.0,3.0,12.0,60.0
27038,1120000,16,0.0,4.0,1.33,70000.0,16.0,1203.007519,2018-07-12,7.0,3.0,12.0,70.0


In [ ]:
# df['new_total_sales'] = df['total_sales']/1000

In [ ]:
df

,total_sales,tickets_sold,tickets_out,show_time,occu_perc,ticket_price,ticket_use,capacity,date,month,quarter,day,new_ticket_price,new_total_sales
0,3900000,26,0.0,4.0,4.26,150000.0,26.0,610.328638,2018-05-05,5.0,2.0,5.0,150.0,3900.0
1,3360000,42,0.0,5.0,8.08,80000.0,42.0,519.801980,2018-05-05,5.0,2.0,5.0,80.0,3360.0
2,2560000,32,0.0,4.0,20.00,80000.0,32.0,160.000000,2018-05-05,5.0,2.0,5.0,80.0,2560.0
3,1200000,12,0.0,1.0,11.01,100000.0,12.0,108.991826,2018-05-05,5.0,2.0,5.0,100.0,1200.0
4,1200000,15,0.0,3.0,16.67,80000.0,15.0,89.982004,2018-05-05,5.0,2.0,5.0,80.0,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27035,1200000,20,0.0,3.0,2.75,60000.0,20.0,727.272727,2018-07-12,7.0,3.0,12.0,60.0,1200.0
27036,1200000,15,0.0,8.0,3.68,80000.0,15.0,407.608696,2018-07-12,7.0,3.0,12.0,80.0,1200.0
27037,1140000,19,0.0,5.0,4.37,60000.0,19.0,434.782609,2018-07-12,7.0,3.0,12.0,60.0,1140.0
27038,1120000,16,0.0,4.0,1.33,70000.0,16.0,1203.007519,2018-07-12,7.0,3.0,12.0,70.0,1120.0


In [ ]:

price_freq = df['new_ticket_price'].value_counts()

price_freq.head()

80.0     4201
60.0     2611
50.0     2277
100.0    2245
150.0    2201
Name: new_ticket_price, dtype: int64

In [ ]:
price_freq.tail()

38.347107     1
40.572917     1
28.909091     1
128.864407    1
82.653061     1
Name: new_ticket_price, dtype: int64

In [ ]:
X = df.drop(columns=['ticket_price','new_ticket_price','total_sales','tickets_out','date'])
y = df['new_ticket_price'].values

In [ ]:
print(X)
print(y)

       tickets_sold  show_time  occu_perc  ticket_use     capacity  month  \
0                26        4.0       4.26        26.0   610.328638    5.0   
1                42        5.0       8.08        42.0   519.801980    5.0   
2                32        4.0      20.00        32.0   160.000000    5.0   
3                12        1.0      11.01        12.0   108.991826    5.0   
4                15        3.0      16.67        15.0    89.982004    5.0   
...             ...        ...        ...         ...          ...    ...   
27035            20        3.0       2.75        20.0   727.272727    7.0   
27036            15        8.0       3.68        15.0   407.608696    7.0   
27037            19        5.0       4.37        19.0   434.782609    7.0   
27038            16        4.0       1.33        16.0  1203.007519    7.0   
27039            16        2.0       7.27        16.0   220.082531    7.0   

       quarter   day  new_total_sales  
0          2.0   5.0           3900

In [ ]:
# encoder = OneHotEncoder()
# X = encoder.fit_transform(X_df.values)


In [ ]:
# for category in encoder.categories_:
#     print(category[:5])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=2019
)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16219, 9)
(10814, 9)
(16219,)
(10814,)


In [ ]:
%%time
model = LinearRegression()
model.fit(X_train, y_train)

CPU times: user 6.2 ms, sys: 9.21 ms, total: 15.4 ms
Wall time: 29.6 ms


In [ ]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("Train Score:", train_score)
print("Test Score:", test_score)

Train Score: 0.32487450692399267
Test Score: 0.3337615768346416


In [ ]:
import math
def compute_mse(model, X, y_true, name):
    y_pred = model.predict(X)
    mse = mean_squared_error(y_true, y_pred)
    mse = math.sqrt(mse)
    mse = math.sqrt(mse)
    print(f'Mean Squared Error for {name}: {mse}')
    
compute_mse(model, X_train, y_train, 'training set')
compute_mse(model, X_test, y_test, 'test set')

Mean Squared Error for training set: 5.291187327302365
Mean Squared Error for test set: 5.33032377724459


In [ ]:
def build_evaluate_fn(X_train, y_train, X_test, y_test):
    def evaluate(model):
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)

        print("Train Score:", train_score)
        print("Test Score:", test_score)
        print()
        
        compute_mse(model, X_train, y_train, 'training set')
        compute_mse(model, X_test, y_test, 'test set')
    
    return evaluate

evaluate = build_evaluate_fn(X_train, y_train, X_test, y_test)

In [ ]:
%%time
svm = LinearSVR()
svm.fit(X_train, y_train);

CPU times: user 948 ms, sys: 0 ns, total: 948 ms
Wall time: 946 ms


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
evaluate(svm)

Train Score: -143.14842894603106
Test Score: -112.28410578943476

Mean Squared Error for training set: 20.225976061763518
Mean Squared Error for test set: 19.248124375514077
